<div style="background-color: lightgreen; border-radius: 5px; padding: 10px;">
    <h4>Collecting Text Window</h4>
    <p>...</p>
</div>

### Imports

In [1]:
import pandas as pd
from itertools import combinations

import time
import warnings
from tqdm.notebook import tqdm
import os

from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re

In [ ]:
# !pip install pandas
# !pip install nltk
# !pip install ipywidgets==7.7.0

In [8]:
# Settings that can be changed
BASEFOLDER = '../../../../../data/enwiki_extracted/'
cities_df = pd.read_csv(f'../../../input/List_of_cities_300k.csv', delimiter=';')
window_size = 0 # Set to Paragraph

# corpus_name = 'city_matrix.csv'
# new_corpus = False

### City Pairs Information

In [160]:
list_of_cities = list(cities_df['Mua_en'][0:150].str.lower())

city_pair_dict = {}
old_city_pairs = []
cumulative_city_pairs = 0
group_size = 5
dataframe_rows = []

for i in range(0, 150, group_size):
    total_city_pairs = list(combinations(list_of_cities[0:i + group_size], 2))
    new_city_pairs = list(set(total_city_pairs) - set(old_city_pairs))  
    
    city_pair_dict[i+5] = new_city_pairs
    
    cumulative_city_pairs += len(new_city_pairs)
    new_cities = list_of_cities[i:i + group_size] 
    
    dataframe_rows.append([i+5, len(new_city_pairs), cumulative_city_pairs, new_cities])
    
    old_city_pairs = total_city_pairs
    
df = pd.DataFrame(dataframe_rows,columns=['cities', 'extra city pairs', 'cumulative city pairs', 'new cities'])
df.head(20)

,cities,extra city pairs,cumulative city pairs,new cities
0,5,10,10,"[paris, london, madrid, berlin, milan]"
1,10,35,45,"[barcelona, athens, rome, birmingham, lisbon]"
2,15,60,105,"[naples, katowice, manchester, hamburg, budapest]"
3,20,85,190,"[bucharest, warsaw, stuttgart, vienna, munich]"
4,25,110,300,"[brussels, stockholm, frankfurt, cologne, cope..."
5,30,135,435,"[valencia, turin, glasgow, prague, lyon]"
6,35,160,595,"[sofia, liverpool, porto, seville, dublin]"
7,40,185,780,"[helsinki, amsterdam, rotterdam, düsseldorf, e..."
8,45,210,990,"[lille, lodz, marseille, antwerp, bilbao]"
9,50,235,1225,"[newcastle, krakow, bochum-herne, thessaloniki..."


### Select city groups to add

In [43]:
OUTPUT_PATH = '../../../../../data/clean/enwiki_city_pairs/'
cities_to_add = [45,50, 55, 60]
override = False

for group in cities_to_add:
    if group % group_size != 0:
        raise Exception("You are expected to add 5 cities at a time, so the groups need to be divisible by 5.")

accepted_cities_to_add = []
existing_groups = []

for group in cities_to_add:
    group_dir = os.path.join(OUTPUT_PATH, str(group))
    if os.path.exists(group_dir) and not override:
        print(f"Group {group} already seems to exist and will be skipped.")
        existing_groups.append(group)
    elif override:
        existing_groups.append(group)
        accepted_cities_to_add.append(group)
    else:
        accepted_cities_to_add.append(group)

accepted_cities_to_add = sorted(accepted_cities_to_add)
city_pairs_to_process = {group: city_pair_dict[group] for group in accepted_cities_to_add}

print(f"\nThe following already exist: {existing_groups} groups and {df[df.cities.isin(existing_groups)]['extra city pairs'].sum()} city pairs already exist.")
print(f"The following will be added: {accepted_cities_to_add} groups and {df[df.cities.isin(accepted_cities_to_add)]['extra city pairs'].sum()} city pairs will be added.")


The following already exist: [] groups and 0 city pairs already exist.
The following will be added: [45, 50, 55, 60] groups and 990 city pairs will be added.


In [3]:
# OUTPUT_PATH = '../../../../data/enwiki_city_pairs/biggest_cities_50/'
# updated_nr_of_cities = 50 # by population numbers
# # current_nr_of_cities = 40

# if updated_nr_of_cities % 5 != 0:
#     raise Exception("You are expected to add 5 cities at a time, so the number of cities needs to be divisible by 5.")

# list_of_cities = list(cities_df['Mua_en'][0:updated_nr_of_cities].str.lower())

# city_pair_dict = {}
# old_city_pairs = []
# cumulative_city_pairs = 0

# dataframe_rows = []

# for i in range(0, len(list_of_cities), 5):
    
#     total_city_pairs = list(combinations(list_of_cities[0:i + n], 2))
#     new_city_pairs = list(set(total_city_pairs) - set(old_city_pairs))  
    
#     city_pair_dict[i+5] = new_city_pairs
    
#     cumulative_city_pairs += len(new_city_pairs)
#     new_cities = list_of_cities[i:i + n] 
    
#     dataframe_rows.append([i+5, len(new_city_pairs), cumulative_city_pairs, new_cities])
    
#     old_city_pairs = total_city_pairs
    
# df = pd.DataFrame(dataframe_rows,columns=['cities', 'extra city pairs', 'cumulative city pairs', 'new cities'])

# groups_to_process = []
# existing_groups = []

# for group in city_pair_dict.keys():
#     if not os.path.exists(f'../../../../../data/clean/enwiki_city_pairs/{str(group)}/'):
#         groups_to_process.append(group)
#     else:
#         existing_groups.append(group)

# new_groups_df = df[df.cities.isin(groups_to_process)]
# existing_groups_df = df[df.cities.isin(existing_groups)]

# print(f"The following groups already exist: {existing_groups}. {existing_groups_df['extra city pairs'].sum()} city pairs already exist.")
# print(f"The following groups will be added: {groups_to_process}. {new_groups_df['extra city pairs'].sum()} city pairs will be added. ")

# city_pairs_to_process = {group: city_pair_dict[group] for group in groups_to_process}

In [47]:
updated_list_of_cities = list(cities_df['Mua_en'][0:updated_nr_of_cities].str.lower())
updated_city_pairs = list(combinations(updated_list_of_cities, 2))
updated_list_of_cities

['paris',
 'london',
 'madrid',
 'berlin',
 'milan',
 'barcelona',
 'athens',
 'rome',
 'birmingham',
 'lisbon',
 'naples',
 'katowice',
 'manchester',
 'hamburg',
 'budapest',
 'bucharest',
 'warsaw',
 'stuttgart',
 'vienna',
 'munich',
 'brussels',
 'stockholm',
 'frankfurt',
 'cologne',
 'copenhagen',
 'valencia',
 'turin',
 'glasgow',
 'prague',
 'lyon',
 'sofia',
 'liverpool',
 'porto',
 'seville',
 'dublin',
 'helsinki',
 'amsterdam',
 'rotterdam',
 'düsseldorf',
 'essen-oberhausen',
 'lille',
 'lodz',
 'marseille',
 'antwerp',
 'bilbao',
 'newcastle',
 'krakow',
 'bochum-herne',
 'thessaloniki',
 'nuremberg',
 'riga',
 'duisburg',
 'dortmund',
 'hanover',
 'zürich']

In [4]:
# my_list = temp_dict.values()

# flat_list = [num for sublist in my_list for num in sublist]
# print(len(flat_list))

In [5]:
# test_list = list(cities_df['Mua_en'][0:60].str.lower())
# n=5
# output={i+5:list(combinations(test_list[0:i + n], 2)) for i in range(0, len(test_list), n)}
# # print(output)

# for key in output:
#     if key < 30:
#         print(key, len(output[key]))

In [6]:
# for group, cities in output.items():
#     print(group, cities)

In [43]:
current_list_of_cities = list(cities_df['Mua_en'][0:current_nr_of_cities].str.lower())
current_city_pairs = list(combinations(current_list_of_cities, 2))

In [44]:
print(f'''Adding '{updated_nr_of_cities-current_nr_of_cities}' cities --> From {current_nr_of_cities} to {updated_nr_of_cities} cities.
Adding '{len(updated_city_pairs)-len(current_city_pairs)}' city pairs --> From {len(current_city_pairs)} to {len(updated_city_pairs)} city pairs.
''')

Adding '10' cities --> From 0 to 10 cities.
Adding '45' city pairs --> From 0 to 45 city pairs.



In [7]:
CITY_PAIRS = [item for item in updated_city_pairs if item not in current_city_pairs]

NameError: name 'updated_city_pairs' is not defined

In [25]:
BASEFOLDER = '../../../../../data/enwiki_extracted/'
OUTPUT_PATH = '../../../../../data/clean/enwiki_city_pairs/'

In [110]:
# test_list = list(cities_df['Mua_en'][0:60].str.lower())

# temp_dict = {}
# old_city_pairs = []
# for i in range(0, len(test_list), n):
#     total_city_pairs = list(combinations(test_list[0:i + n], 2))
#     new_city_pairs = list(set(total_city_pairs) - set(old_city_pairs))
#     print(i+5, len(new_city_pairs), new_city_pairs[:2])
#     temp_dict[i+5] = new_city_pairs
#     old_city_pairs = list(combinations(test_list[0:i + n], 2))

5 10 [('paris', 'london'), ('london', 'berlin')]
10 35 [('berlin', 'athens'), ('barcelona', 'lisbon')]
15 60 [('paris', 'budapest'), ('madrid', 'katowice')]
20 85 [('naples', 'bucharest'), ('manchester', 'bucharest')]
25 110 [('munich', 'cologne'), ('naples', 'brussels')]
30 135 [('frankfurt', 'turin'), ('warsaw', 'glasgow')]
35 160 [('milan', 'liverpool'), ('sofia', 'dublin')]
40 185 [('lyon', 'essen-oberhausen'), ('madrid', 'düsseldorf')]
45 210 [('naples', 'antwerp'), ('manchester', 'antwerp')]
50 235 [('helsinki', 'newcastle'), ('stuttgart', 'thessaloniki')]
55 260 [('naples', 'riga'), ('rotterdam', 'zürich')]
60 285 [('helsinki', 'oslo'), ('glasgow', 'bremen')]


In [126]:
paragraphs_df = pd.DataFrame(columns= ['city_pair', 'paragraph_id', 'paragraph', 'article_id', 'title', 'url'])
# paragraphs_df.to_csv("..\..\..\..\..\data\clean\paragraphs_raw.csv", index=False)

In [139]:
# paragraphs_df

In [152]:
# [[2], [3]]+ [[1], [3]]

paragraphs_df = pd.read_csv(f"..\..\..\..\..\data\clean\paragraphs\paragraphs_raw_folder_{1}.csv")
paragraphs_df.head()

,city_pair,paragraph_id,paragraph,article_id,title,url
0,"('naples', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
1,"('milan', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
2,"('rome', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
3,"('berlin', 'dresden')",1,He shortened his stay in Italy because of the ...,700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
4,"('stuttgart', 'nuremberg')",1,"After his tenure in academia, he continued to ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700


In [149]:
# paragraphs_df['paragraph'][10]

# paragraphs_df = paragraphs_df
# Create subdirectories in output path
# [os.makedirs(os.path.join(output_dir, str(group)), exist_ok=override) for group in city_pairs.keys()]

In [155]:
from IPython.display import clear_output
from ipywidgets import Output
from IPython.display import display
out = Output()
display(out)

import sys
tokenizer = RegexpTokenizer('\w+')

def process_wikidump(city_pairs, basefolder=BASEFOLDER, output_dir=OUTPUT_PATH, override=False, df=paragraphs_df):
    global paragraphs_df
    
    warnings.simplefilter('ignore')
    folders = [f.path for f in os.scandir(basefolder) if f.is_dir()]
    i = 0
    for folder in tqdm(folders, total=len(folders), leave=True, desc='Folders'):
        i+=1
        subfolders = [ f.path for f in os.scandir(folder) if f.is_dir()]
        
        j = 0
        for subfolder in tqdm(subfolders, total=len(subfolders), leave=False, desc='Sub Folders'):
            j+=1
            if '_V6' in subfolder:
                continue
            
            nested_l = []
            # tqdm(os.listdir(subfolder), total=len(os.listdir(subfolder)), leave=True, desc='Text Files'):             
            for file in os.listdir(subfolder):               
                with open(os.path.join(subfolder, file), "r", encoding="utf-8") as f:
                    nested_l = nested_l + process_file(file, f.read(), city_pairs, output_dir, df=paragraphs_df)
                with out:
                    print(subfolder)
                    clear_output(wait=True)
            
            # Changes file name when it has been looked at
            os.replace(subfolder, subfolder + '_V6')
            
            temp_df = pd.DataFrame(data=nested_l,columns=paragraphs_df.columns)
            paragraphs_df = pd.concat([paragraphs_df, temp_df], axis=0, ignore_index=True)
            
            print(len(paragraphs_df))
            paragraphs_df.to_csv(f"..\..\..\..\..\data\clean\paragraphs\paragraphs_raw_folder_{i}.csv", index=False)
            
    return paragraphs_df


def process_file(filename, file_content, city_pairs, output_dir, df):
    articles = file_content.split('</doc>')
    articles = [x.strip() for x in articles if len(x.strip())]    
    nested_l = []
    
    for idx, article in enumerate(articles):
        for paragraph in article.split('\n'):
            tokenized_paragraph = tokenizer.tokenize(paragraph.lower())
        
            word_count = Counter(tokenized_paragraph)
            words = set(word_count.keys())
            
            for group, group_city_pairs in city_pairs.items():
                # directory = os.path.join(output_dir, str(group))
                
                for city_pair in group_city_pairs:
                    detected_cities = words.intersection(set(city_pair))

                    if len(detected_cities) == 2:
                        reg_str = "title=\"" + "(.*?)" + "\""
                        title = re.findall(reg_str, article)
                        
                        reg_str = "id=\"" + "(.*?)" + "\""
                        article_id = re.findall(reg_str, article)
                        
                        reg_str = "url=\"" + "(.*?)" + "\""
                        url = re.findall(reg_str, article)
                        
                        paragraph = paragraph.replace('"', "'").replace('“', "'").replace('”', "'")
                        
                        with out:
                            print('\r',filename," || ", "title: ", title[0], " || ", "citypair: ", city_pair, end='                                         ')
                        
                        l = [city_pair, 0, paragraph, article_id[0], title[0], url[0]]
                        nested_l.append([city_pair, idx, paragraph, article_id[0], title[0], url[0]])

    return nested_l
                        
    
    
#                         article_id = find_article_id(line)[11:] # new
#                         # paragraph_id = find_paragraph_id(line)[13:] 

#                        # city_pair = find_city_pair(line)
# #                             
# #                             


#                         content = f'title="{title[0]}", id={article_id[0]} \n{paragraph} \n\n'

#                         textfile = f'{output_dir}{city_pair[0]}_{city_pair[1]}.txt'

# #                         with open(textfile, 'a+', encoding='utf-16') as f:
# #                             f.writelines(content)
                            

#                         if 'title=' in line:
#                         if line.startswith('<doc') or city_pair_paragraphs[index+1].startswith('<doc'):
#                             pass
#                         elif len(city_pair_paragraphs[index+1]) < 80:
#                             pass
#                         else:

#                             title = find_title(line)
#                             city_pair = find_city_pair(line)
#                             article_id = find_article_id(line)[11:]
#                             paragraph_id = find_paragraph_id(line)[13:]

#                             paragraph = city_pair_paragraphs[index+1].replace('"', "'").replace('“', "'").replace('”', "'")

#                             d = { 'Title': title, 'city_pair': city_pair, 'article_id': article_id, 'paragraph_id': paragraph_id, 'paragraph': paragraph}
#                             new_df = pd.DataFrame([d])
#                             df = pd.concat([df, new_df], axis=0, ignore_index=True)

                

Output()

In [156]:
%%time

protection = input(f"Are you sure you want to collect window text of {city_pairs_to_process.keys()} cities? (currently: '...' cities).\nType 'Yes' to continue.")

if protection == 'Y':
    print()
    t = 1
    while t:
        print(f"\rcontinuing processing in {t}...", end="")
        t -= 1
        time.sleep(1)
    clear_output()
    paragraphs_df = process_wikidump(basefolder=BASEFOLDER, output_dir=OUTPUT_PATH, city_pairs=city_pairs_to_process, df=paragraphs_df)
else:
    print("\nCanceling Operation...\n")

Folders:   0%|          | 0/62 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/4 [00:00<?, ?it/s]

Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

3412
4051
4754
5418
6007
6295


Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

...

97140
97822
98412
98956
99439
99818


Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

100413
100915
101447
102064
102798
103028


Sub Folders:   0%|          | 0/6 [00:00<?, ?it/s]

103717
104330
104941
105581
106180
107063
CPU times: total: 8h 13min 48s
Wall time: 8h 20min 10s


In [154]:
paragraphs_df
#paragraphs_df.to_csv(f"..\..\..\..\..\data\clean\paragraphs\paragraphs_raw_folder_{1}.csv", index=False)

,city_pair,paragraph_id,paragraph,article_id,title,url
0,"('naples', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
1,"('milan', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
2,"('rome', 'dresden')",1,"In September 1818, while waiting for his book ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
3,"('berlin', 'dresden')",1,He shortened his stay in Italy because of the ...,700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
4,"('stuttgart', 'nuremberg')",1,"After his tenure in academia, he continued to ...",700,Arthur Schopenhauer,https://en.wikipedia.org/wiki?curid=700
...,...,...,...,...,...,...
2385,"('hamburg', 'bremen')",51,Dominion of England lent the Danes an importan...,40501,Cnut,https://en.wikipedia.org/wiki?curid=40501
2386,"('rome', 'bremen')",51,"Consistent with his role as a Christian king, ...",40501,Cnut,https://en.wikipedia.org/wiki?curid=40501
2387,"('hamburg', 'bremen')",51,"Consistent with his role as a Christian king, ...",40501,Cnut,https://en.wikipedia.org/wiki?curid=40501
2388,"('hamburg', 'bremen')",8,The Archepiscopacies of Hamburg and Bremen had...,40539,Adalbert of Magdeburg,https://en.wikipedia.org/wiki?curid=40539


In [ ]:
%%time

INPUT_DIR = '../../../../../data/enwiki_city_pairs3-raw/'
for root, dirs, files in tqdm(os.walk(INPUT_DIR, topdown=True)):
    
    if root == '../../../../../data/enwiki_city_pairs3-raw/biggest_cities_40':
        print(dirs)
    # Create subdirectories in output path
#     [os.makedirs(os.path.join(OUTPUT_DIR, dir), exist_ok=True) for dir in dirs]

        for name in tqdm(files, total=len(files)):
            file_path = os.path.join(root, name)
            file_output_dir = root.replace(INPUT_DIR, OUTPUT_DIR)
            file_output_dir = '../../../../../data/clean/city_pair_paragraphs3/40'
            
            city_dir = os.path.join(file_output_dir, f"cities___{name[:-4]}___")
            # os.makedirs(city_dir, exist_ok = True)
            
            df = pd.DataFrame(columns= ['Title', 'city_pair', 'article_id', 'paragraph_id', 'paragraph'])

            with open(file_path, 'r', encoding='utf-16') as f:
                city_pair_paragraphs = [x.strip().lower() for x in f.read().split('\n') if len(x)]
                print(len(city_pair_paragraphs)/2)
                for index, line in enumerate(city_pair_paragraphs):
                    if 'title=' in line:
                        if line.startswith('<doc') or city_pair_paragraphs[index+1].startswith('<doc'):
                            pass
                            # print('\n', city_pair_paragraphs[index+1])
                        elif len(city_pair_paragraphs[index+1]) < 80:
                            pass
                        else:

                            title = find_title(line)
                            city_pair = find_city_pair(line)
                            article_id = find_article_id(line)[11:]
                            paragraph_id = find_paragraph_id(line)[13:]

                            paragraph = city_pair_paragraphs[index+1].replace('"', "'").replace('“', "'").replace('”', "'")

                            d = { 'Title': title, 'city_pair': city_pair, 'article_id': article_id, 'paragraph_id': paragraph_id, 'paragraph': paragraph}
                            new_df = pd.DataFrame([d])
                            df = pd.concat([df, new_df], axis=0, ignore_index=True)

                
                df.to_csv(os.path.join(city_dir, f'{name[:-4]}.csv'), index=False)